## Install dependencies

In [4]:
try:
    import fmpy
except:
    !pip install fmpy

## Download files

In [ ]:
from fmpy.util import download_file

download_file(url='https://github.com/t-sommer/fmi-webapp-tutorial/raw/main/Heater.fmu',
              checksum='fe5beced10f7dcf3c3aa453dafa6f60da0f7a5071ad8b2d650bdc7dcc6237246')

## Display text and images

See https://www.markdownguide.org/cheat-sheet/ for more Markdown syntax.

![Heater model](https://github.com/t-sommer/fmi-webapp-tutorial/blob/main/Heater.png?raw=1)

## Get information about the FMU

In [ ]:
from fmpy import dump

dump('Heater.fmu')

## Simulate an FMU and plot the result

In [ ]:
import fmpy
from fmpy import simulate_fmu, plot_result

fmpy.plot_library = 'plotly'  # experimental


filename = 'Heater.fmu'

start_values = {
    # variable                           start   unit      description
    'TAmb':                            (293.15, 'K'),    # Ambient temperature
    'TDif':                                 (2, 'K'),    # Error in temperature
    'constantVoltage.V':                   (10, 'V'),    # Value of constant voltage
    'heatCapacitor.C':                      (1, 'J/K'),  # Heat capacity of element (= cp*m)
    'resistor.R':                          (10, 'Ohm'),  # Resistance at temperature T_ref
    'resistor.T_ref':                  (293.15, 'K'),    # Reference temperature
    'resistor.alpha':     (0.00392156862745098, '1/K'),  # Temperature coefficient of resistance (R_actual = R*(1 + alpha*(T_heatPort - T_ref))
    'thermalConductor.G':                 (0.1, 'W/K'),  # Constant thermal conductance of material
    'idealSwitch.Ron':                  (1E-05, 'Ohm'),  # Closed switch resistance
    'idealSwitch.Goff':                 (1E-05, 'S'),    # Opened switch conductance
}

output = [
    'TRes',  # Resulting temperature
    'T',     # Absolute temperature in degree Celsius as output signal
]

result = simulate_fmu(filename, start_values=start_values, output=output, stop_time=50.0)

plot_result(result)

## Show docstring

In [ ]:
help(dump)

## Open documentation

In [ ]:
dump?

## Access the source code

In [ ]:
dump??